In [ ]:
import FinanceDataReader as fdr
import numpy as np
import matplotlib.pyplot as plt


df = fdr.DataReader('005930', '2011-05-04', '2024-01-22')
# print(df.head())

# print(df.shape)
# print(df.head())


In [ ]:

def MinMaxScaler(data):
    """최솟값과 최댓값을 이용하여 0 ~ 1 값으로 변환"""
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # 0으로 나누기 에러가 발생하지 않도록 매우 작은 값(1e-7)을 더해서 나눔
    return numerator / (denominator + 1e-7)

dfx = df[['Open','High','Low','Volume', 'Close']]
dfx = MinMaxScaler(dfx)

# 종가
dfy = dfx[['Close']]
print("head Close :" , dfy.head())

dfx = dfx[['Open','High','Low','Volume']]
print("head Open:" ,dfx.head())

# print(dfx.describe())

X = dfx.values.tolist()
y = dfy.values.tolist()


# print(X[:5])

In [ ]:
window_size = 10

data_X = []
data_y = []
for i in range(len(y) - window_size):
    _X = X[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
    _y = y[i + window_size]     # 다음 날 종가
    data_X.append(_X)
    data_y.append(_y)
#print(_X, "->", _y)

print(data_X[0])
print(data_y[0])

print('전체 데이터의 크기 :', len(data_X), len(data_y))


In [ ]:
# df.info()
df.sample

In [ ]:

train_size = int(len(data_y) * 0.7)
train_X = np.array(data_X[0 : train_size])
train_y = np.array(data_y[0 : train_size])

test_size = len(data_y) - train_size
test_X = np.array(data_X[train_size : len(data_X)])
test_y = np.array(data_y[train_size : len(data_y)])

print('훈련 데이터의 크기 :', train_X.shape, train_y.shape)
print('테스트 데이터의 크기 :', test_X.shape, test_y.shape)


In [1]:
import sqlparse
import networkx as nx
import matplotlib.pyplot as plt

def parse_sql_query(sql_query):
    """SQL 쿼리 파싱하여 테이블과 조인 정보 추출"""
    parsed = sqlparse.parse(sql_query)[0]
    tables = set()
    joins = set()
    
    for token in parsed.tokens:
        if token.ttype is None:  # From, Join 등의 구조를 찾기 위함
            for item in token.tokens:
                if item.ttype is None and not item.is_whitespace:
                    if item.value.upper() == 'JOIN':
                        joins.add(token.tokens[-1].value)
                    else:
                        tables.add(item.value)
                        
    return tables, joins

def create_graph(tables, joins):
    """테이블과 조인 정보로부터 그래프 생성"""
    G = nx.Graph()
    for table in tables:
        G.add_node(table)
    for join in joins:
        if join in tables:  # 단순화를 위해 테이블이 이미 알려진 경우에만 추가
            G.add_edge(list(tables)[0], join)  # 예제는 단순화를 위한 것으로, 실제 조인 로직에 따라 다름
    return G

def visualize_graph(G):
    """그래프 시각화"""
    nx.draw(G, with_labels=True)
    plt.show()

# 예제 SQL 쿼리
sql_query = "SELECT * FROM Employees JOIN Departments ON Employees.dep_id = Departments.id"

# SQL 쿼리 파싱 및 정보 추출
tables, joins = parse_sql_query(sql_query)

# 그래프 생성 및 시각화
G = create_graph(tables, joins)
visualize_graph(G)


ModuleNotFoundError: No module named 'sqlparse'

In [ ]:
!pip3 install tensorflow
!pip3 install keras
!pip install -U keras_preprocessing --no-deps
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout


model = Sequential()
model.add(LSTM(units=20, activation='relu', return_sequences=True, input_shape=(10, 4)))
model.add(Dropout(0.1))
model.add(LSTM(units=20, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()


In [ ]:

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_X, train_y, epochs=70, batch_size=30)
pred_y = model.predict(test_X)



In [ ]:

plt.figure()
plt.plot(test_y, color='red', label='real SEC stock price')
plt.plot(pred_y, color='blue', label='predicted SEC stock price')
plt.title('SEC stock price prediction')
plt.xlabel('time')
plt.ylabel('stock price')
plt.legend()
plt.show()

print("내일 SEC 주가 :", df.Close[-1] * pred_y[-1] / dfy.Close[-1], 'KRW')

In [ ]:
# python3.8 로 셋팅해야함.
# https://www.tensorflow.org/install/source?hl=ko

